# How to run this notebook

1. Open 2 terminal tabs
2. From Terminal A, ssh into TACC the normal way - `ssh <username>@maverick2.tacc.utexas.edu`
3. From terminal B, ssh into TACC using the command - `ssh -L localhost:8888:127.0.0.1:8888 -L localhost:6006:localhost:6006 sbansal6@maverick2.tacc.utexas.edu`
4. From terminal A, request some resources on the GPU compute resource - `idev -p gtx -L work -m 180`
5. From terminal B, log into the compute resource provisioned using - `ssh -L localhost:8888:127.0.0.1:8888 -L localhost:6006:localhost:6006 <resource-name>`
6. From terminal B, run jupyter notebook - `jupyter lab --port 8888`
7. You should be able to open the jupyter notebook on the browser of your local computer.

# Assumptions

Following are the requirements:

Packages:

```
tensorflow                         2.1.0
tensorflow-estimator               2.1.0
tensorflow-gpu                     2.1.0
tensorflow-probability             0.9.0
tensorboard                        2.3.0
```

Modules:

`module load intel/17.0.4 python3/3.6.3 cuda/10.0 cudnn/7.6.2 nccl/2.4.7 tacc-singularity/3.4.2`

In addition to the above module you might need to add the following to your path:

`export LD_LIBRARY_PATH=$LD_LIBRARY_PATH:/opt/apps/cuda/10.1/targets/x86_64-linux/lib`


Because of some missing library issues in TACC and depending on the missing library you might need to find and add the library somewhere accessible and add that path to the `LD_LIBRARY_PATH`:

```
login1.maverick2(1006)$ find -O3 -L /opt/apps/ -name "*libcupti.so.10.1*"
/opt/apps/cuda/10.1/extras/CUPTI/lib64/libcupti.so.10.1
/opt/apps/cuda/10.1/extras/CUPTI/lib64/libcupti.so.10.1.208

login1.maverick2(1006)$ cp /opt/apps/cuda/10.1/extras/CUPTI/lib64/libcupti.so.10.1 $WORK/libcupti.so.10.1
login1.maverick2(1006)$ export LD_LIBRARY_PATH=$LD_LIBRARY_PATH:$WORK

```

# Experiment Details

There are 2 datasets - 

**A**: orig_faced + defaced volumes

**B**: refaced + defaced volumes


1. Randomly select 49 volumes from faced and refaced datasets. 
2. Generate a new dataset T_A, with these 49 faced + the corresponding 49 defaced, and T_B with the 49 defaced + 49 re-faced.
3. Put T_A and T_B aside
4. Run your 15-fold CV on the remainder:

**A_2 (N = 1000x2)**: 1000 faced + 1000 defaced

**B_2 (N = 1000x2)**: 1000 defaced + 1000 re-faced



## Step1 and Step2

In [ ]:
import os, sys
sys.path.append("..")
import numpy as np
from glob import glob
import pandas as pd
import random
from random import shuffle

# Define paths
ROOT_DIR = '/work/06850/sbansal6/maverick2/mriqc-shared/conformed'

face_path = os.path.join(ROOT_DIR, 'face/128')
defaced_path = os.path.join(ROOT_DIR, 'face_defaced/128')
refaced_path = os.path.join(ROOT_DIR, 'face_refaced/128')

paths_d = []
paths_f = []
paths_r = []

for path in glob(defaced_path + "/*/*.nii*"):
    DS = path.split('/')[-2]
    paths_d.append(path)
    
for path in glob(refaced_path + "/*/*.nii*"):
    DS = path.split('/')[-2]
    paths_r.append(path)
    
for path in glob(face_path + "/*/*.nii*"):
    DS = path.split('/')[-2]
    paths_f.append(path)
    

def generate_datasets(fpaths, dpaths, size, typ ='faced'):
    
    if typ not in ['faced', 'refaced']:
        print("Incorrect value for t. Choose from [faced, refaced]")
        return
    
    random.shuffle(fpaths)
    test_f = fpaths[:size]
    main_f = fpaths[size:]

    test_d = []
    for t in test_f:
        if typ == 'faced':
            test_d.append(t.replace('face', 'face_defaced'))
        
        if typ == 'refaced':
            DS = t.split('/')[-2]
            sub = t.split('/')[-1].replace('_defaced_refaced', '').split('.nii.gz')[0]
            search_pattern = os.path.join(DS, sub)
            
            # match pattern from defaced dataset
            for _d in dpaths:
                if search_pattern in _d:
                    test_d.append(_d)
                

    test = test_f + test_d
    labels_test = [1]*len(test_f) + [0]*len(test_d)
    
    # remove T_A_D from defaced volume set
    main_d = list(set(dpaths) - set(test_d))
    
    labels_main = [1]*len(main_f) + [0]*len(main_d)
    main = main_f + main_d
    
    return main, labels_main, test, labels_test

A_2, L_A_2, T_A, L_T_A = generate_datasets(paths_f, paths_d, 49, typ='faced')
B_2, L_B_2, T_B, L_T_B = generate_datasets(paths_r, paths_d, 49, typ='refaced')

print(len(A_2), len(T_A))
print(len(B_2), len(T_B))


## Generate n-fold CV Datasets

In [ ]:
from operator import itemgetter
from sklearn.model_selection import KFold
from sklearn.model_selection import StratifiedKFold
from sklearn.model_selection import train_test_split
import pandas as pd
import random
from random import shuffle
import os

def generate_CSV(paths, labels, save_path, test_paths=None, test_labels=None, n=15, mode='CV'):
    
    os.makedirs(save_path, exist_ok=True)
    
    df = pd.DataFrame()
    df["X"] = paths
    df["Y"] = labels
    df.to_csv(os.path.join(save_path, "all.csv"))
    
    if mode == 'CV':
        SPLITS = n
        skf = StratifiedKFold(n_splits=SPLITS)
        fold_no = 1

        for train_index, test_index in skf.split(paths, labels):
            out_path = os.path.join(save_path, "train_test_fold_{}/csv/".format(fold_no))

            if not os.path.exists(out_path):
                os.makedirs(out_path)

            image_train, image_test = (
                itemgetter(*train_index)(paths),
                itemgetter(*test_index)(paths),
            )

            label_train, label_test = (
                itemgetter(*train_index)(labels),
                itemgetter(*test_index)(labels),
            )

            train_data = {"X": image_train , "Y": label_train}
            df_train = pd.DataFrame(train_data)
            df_train.to_csv(os.path.join(out_path, "training.csv"), index=False)

            validation_data = {"X": image_test, "Y": label_test}
            df_validation = pd.DataFrame(validation_data)
            df_validation.to_csv(os.path.join(out_path, "validation.csv"), index=False)

            fold_no += 1
    else:
        train_data = {"X": paths , "Y": labels}
        df_train = pd.DataFrame(train_data)
        df_train.to_csv(os.path.join(save_path, "training.csv"), index=False)
        
        test_data = {"X": test_paths , "Y": test_labels}
        df_test = pd.DataFrame(test_data)
        df_test.to_csv(os.path.join(save_path, "testing.csv"), index=False)
        
## CROSS VALIDATION
# generate_CSV(A_2, L_A_2, "experiments/experiment_A/csv_F15")
# generate_CSV(B_2, L_B_2, "/work/06850/sbansal6/maverick2/mriqc-shared/experiments/experiment_B/csv_F15")


## DEFINE A ROOT DIR where all the data will be stored <<<<<
ROOT_DIR = '/work/06850/sbansal6/maverick2/mriqc-shared/experiments' 

## FULL DATASET
generate_CSV(A_2,
             L_A_2,
             os.path.join(ROOT_DIR, 'experiment_A/128/csv_full'),
             test_paths=T_A,
             test_labels=L_T_A,
             mode='full')

generate_CSV(B_2,
             L_B_2,
             os.path.join(ROOT_DIR, 'experiment_B/128/csv_full'),
             test_paths=T_B,
             test_labels=L_T_B,
             mode='full')


# Generate tfrecords for n-fold CV datasets

In [ ]:
import random
import nobrainer
import os, sys
sys.path.append("..")
import numpy as np
import nibabel as nb
from glob import glob
from pathlib import Path
from shutil import *
import subprocess
from operator import itemgetter
import pandas as pd


def generate_tfrecords(csv_path, records_save_path, mode='CV'):
    
    os.makedirs(records_save_path, exist_ok=True)

    train_csv_path = os.path.join(csv_path, "training.csv")
    

    train_paths = pd.read_csv(train_csv_path)["X"].values
    train_labels = pd.read_csv(train_csv_path)["Y"].values
    train_D = list(zip(train_paths, train_labels))
    
    random.shuffle(train_D)

    train_write_path = os.path.join(records_save_path, 'data-train_shard-{shard:03d}.tfrec')
    
    nobrainer.tfrecord.write(
        features_labels=train_D,
        filename_template=train_write_path,
        examples_per_shard=3)
    
    if mode =='CV':
        valid_csv_path = os.path.join(csv_path, "validation.csv")
        valid_paths = pd.read_csv(valid_csv_path)["X"].values
        valid_labels = pd.read_csv(valid_csv_path)["Y"].values
        valid_D = list(zip(valid_paths, valid_labels))
        random.shuffle(valid_D)
        valid_write_path = os.path.join(records_save_path, 'data-valid_shard-{shard:03d}.tfrec')

        nobrainer.tfrecord.write(
            features_labels=valid_D,
            filename_template=valid_write_path,
            examples_per_shard=1)
    
    if mode == 'test':
        test_csv_path = os.path.join(csv_path, "testing.csv")
        test_paths = pd.read_csv(test_csv_path)["X"].values
        test_labels = pd.read_csv(test_csv_path)["Y"].values
        test_D = list(zip(test_paths, test_labels))
        random.shuffle(test_D)
        test_write_path = os.path.join(records_save_path, 'data-test_shard-{shard:03d}.tfrec')

        nobrainer.tfrecord.write(
            features_labels=test_D,
            filename_template=test_write_path,
            examples_per_shard=1)

## Cross-Validation 
# SPLITS = 15
# for fold in range(1, SPLITS+1):
#     print("FOLD: ", fold)
#     csv_path = "/work/06850/sbansal6/maverick2/mriqc-shared/experiments/experiment_B/csv_F15/train_test_fold_{}/csv/".format(fold)
#     tf_records_dir = "/work/06850/sbansal6/maverick2/mriqc-shared/experiments/experiment_B/tfrecords_F15/tfrecords_fold_{}/".format(fold)
#     generate_tfrecords(csv_path, tf_records_dir)


# Test (full dataset)
# experiment_A
# csv_path = os.path.join(ROOT_DIR, "experiment_A/128/csv_full")
# tf_records_dir = os.path.join(ROOT_DIR, "experiment_A/128/tfrecords_full")
# generate_tfrecords(csv_path, tf_records_dir, mode='test')

# experiment_B
csv_path = os.path.join(ROOT_DIR, "experiment_B/128/csv_full")
tf_records_dir = os.path.join(ROOT_DIR, "experiment_B/128/tfrecords_full")
generate_tfrecords(csv_path, tf_records_dir, mode='test')

## Main held-out Test Dataset

In [1]:
!pip install sklearn pandas

     |████████████████████████████████| 9.5 MB 4.1 MB/s eta 0:00:01
     |████████████████████████████████| 22.2 MB 374 kB/s  eta 0:00:01
     |████████████████████████████████| 510 kB 12.0 MB/s eta 0:00:01
     |████████████████████████████████| 302 kB 8.6 MB/s eta 0:00:01
  Created wheel for sklearn: filename=sklearn-0.0-py2.py3-none-any.whl size=2397 sha256=0320d989c399dc67c56e1ec17579099f3f9bef3c5852c1f10449eced239cb894
  Stored in directory: /root/.cache/pip/wheels/23/9d/42/5ec745cbbb17517000a53cecc49d6a865450d1f5cb16dc8a9c
Successfully built sklearn
You should consider upgrading via the '/usr/bin/python3 -m pip install --upgrade pip' command.


In [2]:
import nobrainer

## Initialize dataset model

In [1]:
import nobrainer
from nobrainer.io import _is_gzipped
from nobrainer.volume import to_blocks
import sys, os
sys.path.append('../defacing')
from preprocessing.augmentation import VolumeAugmentations, SliceAugmentations
from helpers.utils import load_vol
import tensorflow as tf
import glob
import numpy as np

AUTOTUNE = tf.data.experimental.AUTOTUNE
DISTRIBUTION = load_vol('../defacing/helpers/distribution.nii.gz')[0]
DISTRIBUTION /= DISTRIBUTION.sum()
COM = np.unravel_index(int(np.sum(DISTRIBUTION.ravel()*np.arange(len(DISTRIBUTION.ravel())))/np.sum(DISTRIBUTION.ravel())), DISTRIBUTION.shape)


# sampling from augmented distribution is same as augmenting the sampled points
# augmenting distribution at every iteration is expensive, so this way
sampler = lambda n_slices, distribution = DISTRIBUTION, threshold = 0.1: np.array([ np.unravel_index(
          np.random.choice(np.arange(np.prod(distribution.shape)),
                                     p = distribution.ravel()),
          distribution.shape) + (+1 if np.random.randn() > 0.5 else -1)*np.random.randint(0, 
                                        int(distribution.shape[0]*threshold) + 1, 3) for _ in range(n_slices)]) 


three_d_augmentations = {'rotation': 0.5,
                         'translation': 0.5,
                         'noop': 0.3
                        }

augmentvolume = VolumeAugmentations(DISTRIBUTION, three_d_augmentations)

two_d_augmentations = {'rotation': 0.5,
                       'fliplr': 0.5,
                       'flipud': 0.5,
                       'zoom': 0.5,
                       'noop': 0.3
                      }

# augmentslice = VolumeAugmentations(DISTRIBUTION, two_d_augmentations)


def get_dataset(
    file_pattern,
    n_classes,
    batch_size,
    volume_shape,
    plane,
    n_slices = 24,
    block_shape=None,
    n_epochs=None,
    mapping=None,
    augment=False,
    shuffle_buffer_size=None,
    num_parallel_calls=AUTOTUNE,
    mode='train',
):

    """ Returns tf.data.Dataset after preprocessing from
    tfrecords for training and validation

    Parameters
    ----------
    file_pattern:

    n_classes:
    """

    files = glob.glob(file_pattern)

    if not files:
        raise ValueError("no files found for pattern '{}'".format(file_pattern))

    compressed = _is_gzipped(files[0])
    shuffle = bool(shuffle_buffer_size)

    ds = nobrainer.dataset.tfrecord_dataset(
        file_pattern=file_pattern,
        volume_shape=volume_shape,
        shuffle=shuffle,
        scalar_label=True,
        compressed=compressed,
        num_parallel_calls=num_parallel_calls,
    )

    # if augment:
    #     ds = ds.map(
    #         lambda x, y: tf.cond(
    #             tf.random.uniform((1,)) > 0.5,
    #             true_fn=lambda: apply_augmentations(x, y),
    #             false_fn=lambda: (x, y),
    #         ),
    #         num_parallel_calls=num_parallel_calls,
    #     )

    def _ss(x, y):
        if augment:
            if three_d_augmentations['noop'] < 1:
                x, y = augmentvolume(x,y)
        x, y = structural_slice(x, y, 
                                plane, 
                                n_slices, 
                                augment, 
                                augmentvolume.distribution)
        return (x, y)
    
    
    ds = ds.map(_ss, num_parallel_calls)
    
    ds = ds.prefetch(buffer_size=batch_size)

    if batch_size is not None:
        ds = ds.batch(batch_size=batch_size, drop_remainder=True)
        
    if mode == 'train':
        if shuffle_buffer_size:
            ds = ds.shuffle(buffer_size=shuffle_buffer_size)

        # Repeat the dataset n_epochs times
        ds = ds.repeat(n_epochs)

    return ds


def structural_slice(x, y, 
                plane, 
                n_slices = 4, 
                augment = False, 
                distribution = DISTRIBUTION):

    """ Transpose dataset based on the plane

    Parameters
    ----------
    x:

    y:

    plane:
    
    n:

    augment:
    """

    threshold = 0.1 if augment else 0.0 
    options = ["axial", "coronal", "sagittal", "combined"]
    shape = np.array(x.shape)

    if isinstance(plane, str) and plane in options:
        idxs = sampler(n_slices, 
                        distribution, 
                        threshold)

        if plane == "axial":
            idx = np.random.randint(shape[0]**0.5)
            midx = idxs[:, 0]
            x = x

        if plane == "coronal":
            idx = np.random.randint(shape[1]**0.5)
            midx = idxs[:, 1]
            x = tf.transpose(x, perm=[1, 2, 0])


        if plane == "sagittal":
            idx = np.random.randint(shape[2]**0.5)
            midx = idxs[:, 2]
            x = tf.transpose(x, perm=[2, 0, 1])


        if plane == "combined":
            temp = {}
            for op in options[:-1]:
                temp[op] = structural_slice(x, y, 
                                            op, 
                                            n_slices, 
                                            augment, 
                                            distribution)[0]
            x = temp

        if not plane == "combined":
            x = tf.squeeze(tf.gather_nd(x, midx.reshape(n_slices, 1, 1)), axis=1)
            x = tf.math.reduce_mean(x, axis=0)
            x = tf.expand_dims(x, axis=-1)
            
            if augment:
                x = two_d_augmentations(x)
                
            x = tf.convert_to_tensor(x)
        return x, y
    else:
        raise ValueError("expected plane to be one of ['axial', 'coronal', 'sagittal']")


if __name__ == "__main__":
    ROOTDIR = '/tf/shank/HDDLinux/Stanford/data/mriqc-shared/experiments/experiment_A/128/tfrecords_full'
    n_classes = 2
    global_batch_size = 8
    volume_shape = (64, 64, 64)
    ds = get_dataset(
        os.path.join(ROOTDIR, "data-train_*"),
        n_classes=n_classes,
        batch_size=global_batch_size,
        volume_shape=volume_shape,
        plane="sagittal",
        augment = False,
        shuffle_buffer_size=3,
    )

#     import matplotlib
#     matplotlib.use('Agg')
#     import matplotlib.pyplot as plt

    print(ds)


<RepeatDataset shapes: ((8, 64, 64, 1), (8, 1)), types: (tf.float32, tf.float32)>


In [ ]:
!pip install pandas

## Initialize model

In [2]:
# Std packages
import sys, os
import glob
import math

sys.path.append("../defacing")

# Custom packages
from models import modelN
# from dataloaders.dataset import get_dataset

# Tf packages
import tensorflow as tf
import pandas as pd
import numpy as np
from sklearn.utils import class_weight
from tensorflow.keras import backend as K
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import (
    ModelCheckpoint,
    LearningRateScheduler,
    TensorBoard,
)
from tensorflow.keras import metrics
from tensorflow.keras import losses


def scheduler(epoch):
    if epoch < 3:
        return 0.001
    else:
        return 0.001 * tf.math.exp(0.1 * (10 - epoch))


def train(
    csv_path,
    model_save_path,
    tfrecords_path,
    volume_shape=(64, 64, 64),
    image_size=(64, 64),
    dropout=0.2,
    batch_size=16,
    n_classes=2,
    n_epochs=15,
    percent=100,
    mode='CV',
):
    
    
    train_csv_path = os.path.join(csv_path, "training.csv")
    train_paths = pd.read_csv(train_csv_path)["X"].values
    train_labels = pd.read_csv(train_csv_path)["Y"].values
    
    if mode == 'CV':
        valid_csv_path = os.path.join(csv_path, "validation.csv")
        valid_paths = pd.read_csv(valid_csv_path)["X"].values
        valid_labels = pd.read_csv(valid_csv_path)["Y"].values
    
    weights = class_weight.compute_class_weight('balanced',
                                                np.unique(train_labels),
                                                train_labels)
    weights = dict(enumerate(weights))
    
    print(weights)
    
    planes = ["axial", "coronal", "sagittal", "combined"]
    

    global_batch_size = batch_size
    
    os.makedirs(model_save_path, exist_ok=True)
    cp_save_path = os.path.join(model_save_path, "weights")

    logdir_path = os.path.join(model_save_path, "tb_logs")
    if not os.path.exists(logdir_path):
        os.makedirs(logdir_path)

    for plane in planes:

        logdir = os.path.join(logdir_path, plane)
        os.makedirs(logdir, exist_ok=True)

        tbCallback = TensorBoard(
            log_dir=logdir, histogram_freq=1, write_graph=True, write_images=True,
        )

        os.makedirs(os.path.join(cp_save_path, plane), exist_ok=True)

        model_checkpoint = ModelCheckpoint(
            os.path.join(cp_save_path, plane, "best-wts.h5"),
            monitor="val_loss",
            save_weights_only=True,
            mode="min",
        )

#         with strategy.scope():

        if not plane == "combined": 
            lr = 1e-3
            model = modelN.Submodel(
                input_shape=image_size,
                dropout=dropout,
                name=plane,
                include_top=True,
                weights=None,
            )
        else:
            lr = 5e-4
            model = modelN.CombinedClassifier(
                input_shape=image_size,
                dropout=dropout,
                trainable=True,
                wts_root=cp_save_path,
            )

        print("Submodel: ", plane)
#         print(model.summary())

        METRICS = [
            metrics.TruePositives(name="tp"),
            metrics.FalsePositives(name="fp"),
            metrics.TrueNegatives(name="tn"),
            metrics.FalseNegatives(name="fn"),
            metrics.BinaryAccuracy(name="accuracy"),
            metrics.Precision(name="precision"),
            metrics.Recall(name="recall"),
            metrics.AUC(name="auc"),
        ]

        model.compile(
            loss=tf.keras.losses.binary_crossentropy,
            optimizer=Adam(learning_rate=lr),
            metrics=METRICS,
        )

        print("GLOBAL BATCH SIZE: ", global_batch_size)

        dataset_train = get_dataset(
            file_pattern=os.path.join(tfrecords_path, 'data-train_*'),
            n_classes=n_classes,
            batch_size=global_batch_size,
            volume_shape=volume_shape,
            plane=plane,
            shuffle_buffer_size=global_batch_size,
        )
        
        steps_per_epoch = math.ceil(len(train_paths)/batch_size)
        print(steps_per_epoch)
        lrcallback = tf.keras.callbacks.LearningRateScheduler(scheduler)
        
        if mode == 'CV':
            dataset_valid = get_dataset(
                file_pattern=os.path.join(tfrecords_path, "data-valid_*"),
                n_classes=n_classes,
                batch_size=global_batch_size,
                volume_shape=volume_shape,
                plane=plane,
                shuffle_buffer_size=global_batch_size,
            )
            
            validation_steps = math.ceil(len(valid_paths)/batch_size)
            
            model.fit(
                dataset_train,
                epochs=n_epochs,
                steps_per_epoch=steps_per_epoch,
                validation_data=dataset_valid,
                validation_steps=validation_steps,
                callbacks=[tbCallback, model_checkpoint],
                class_weight = weights,
            )
        else:
            print(model.summary())
            print("Steps/Epoch: ", steps_per_epoch)
            model.fit(
                dataset_train,
                epochs=n_epochs,
                steps_per_epoch=steps_per_epoch,
                callbacks=[tbCallback, model_checkpoint],
                class_weight = weights,
            )

        del model
        
        K.clear_session()

## n-fold CV

In [ ]:
ROOT_DIR = '/work/06850/sbansal6/maverick2/mriqc-shared/experiments/experiment_A'

for fold in range(1, 15):
    print("FOLD {}".format(fold))
    csv_path = os.path.join(ROOT_DIR, "csv_F15/train_test_fold_{}/csv".format(fold))
    model_save_path = os.path.join(ROOT_DIR, "model_save_dir_F15/train_test_fold_{}".format(fold))
    tfrecords_path = os.path.join(ROOT_DIR, 'tfrecords_F15/tfrecords_fold_{}'.format(fold))

    train(
        csv_path,
        model_save_path,
        tfrecords_path,
        volume_shape=(128, 128, 128),
        image_size=(128, 128),
        mode='CV'
    )

## Tensorboard graph visualization

NOTE: For this to work, you need to have the 6006 port binded from the localhost as shown in the first block

In [ ]:
# %load_ext tensorboard
# %tensorboard --logdir /work/06850/sbansal6/maverick2/mriqc-shared/experiments/experiment_A/model_save_dir_F15 

## Train a model on full dataset

In [3]:
ROOTDIR = '/tf/shank/HDDLinux/Stanford/data/mriqc-shared/experiments/experiment_B/128'

csv_path = os.path.join(ROOTDIR, "csv_full")
model_save_path = os.path.join(ROOTDIR, "model_save_dir_full_local")
tfrecords_path = os.path.join(ROOTDIR, 'tfrecords_full')


train(
    csv_path,
    model_save_path,
    tfrecords_path,
    volume_shape=(128, 128, 128),
    image_size=(128, 128),
    mode='full'
)

/usr/local/lib/python3.6/dist-packages/sklearn/utils/validation.py:72: FutureWarning: Pass classes=[0 1], y=[1 1 1 ... 0 0 0] as keyword args. From version 1.0 (renaming of 0.25) passing these as positional arguments will result in an error
  "will result in an error", FutureWarning)


{0: 0.9934934934934935, 1: 1.0065922920892494}
Submodel:  axial
GLOBAL BATCH SIZE:  16
125
Model: "model"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
axial (InputLayer)           [(None, 128, 128, 1)]     0         
_________________________________________________________________
conv2d (Conv2D)              (None, 128, 128, 8)       80        
_________________________________________________________________
batch_normalization (BatchNo (None, 128, 128, 8)       32        
_________________________________________________________________
activation (Activation)      (None, 128, 128, 8)       0         
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 128, 128, 8)       584       
_________________________________________________________________
batch_normalization_1 (Batch (None, 128, 128, 8)       32        
____________________________________

125/125 [==============================] - 10s 64ms/step - loss: 1.7148 - tp: 377.7857 - fp: 138.0476 - tn: 369.6032 - fn: 130.4365 - accuracy: 0.6838 - precision: 0.6777 - recall: 0.6882 - auc: 0.7251
Epoch 2/15
 69/125 [===============>..............] - ETA: 3s - loss: 0.1162 - tp: 250.0725 - fp: 11.2464 - tn: 287.8551 - fn: 10.8261 - accuracy: 0.9616 - precision: 0.9567 - recall: 0.9597 - auc: 0.9903

KeyboardInterrupt: 

In [ ]:
import sys, os
sys.path.append('../defacing')
from models.modelN import CombinedClassifier

# Tf packages
import tensorflow as tf
import pandas as pd
import numpy as np
from sklearn.utils import class_weight
from tensorflow.keras import backend as K
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import (
    ModelCheckpoint,
    LearningRateScheduler,
    TensorBoard,
)
from tensorflow.keras import metrics
from tensorflow.keras import losses

ROOTDIR_B = '/work/06850/sbansal6/maverick2/mriqc-shared/experiments/experiment_B/128'
ROOTDIR_A = '/work/06850/sbansal6/maverick2/mriqc-shared/experiments/experiment_A/128'

csv_path = os.path.join(ROOTDIR, "csv_full")
model_save_path = os.path.join(ROOTDIR_B, "model_save_dir_full")
tfrecords_path = os.path.join(ROOTDIR_B, 'tfrecords_full')
weights_path = os.path.join(model_save_path, 'weights/combined/best-wts.h5')

planes = ['axial', 'coronal', 'sagittal']
for plane in planes:
    model = modelN.Submodel(
        input_shape=image_size,
        dropout=dropout,
        name=plane,
        include_top=True,
         weights=None)
    
    model.load_weights(os.path.join(weights_path, ))
    
    
    
model = CombinedClassifier(
    input_shape=(128, 128), dropout=0.4, wts_root=None, trainable=True
)
model.load_weights(os.path.abspath(weights_path))


dataset_test = get_dataset(
    file_pattern=os.path.join(tfrecords_path, "data-test_*"),
    n_classes=2,
    n_slices = 24,
    batch_size=16,
    volume_shape=(128, 128, 128),
    plane='combined',
    mode='test'
)

METRICS = [
            metrics.BinaryAccuracy(name="accuracy"),
            metrics.Precision(name="precision"),
            metrics.Recall(name="recall"),
            metrics.AUC(name="auc"),
        ]

model.compile(
    loss=tf.keras.losses.binary_crossentropy,
    optimizer=Adam(learning_rate=1e-3),
    metrics=METRICS,
)

    
results = model.evaluate(dataset_test)
predictions = (model.predict(dataset_test) > 0.5).astype(int)

In [ ]:
print(dataset_test)
import matplotlib.pyplot as plt
corr_pred_map = {}

# for x, y in dataset_test.enumerate():
for x, y in dataset_test.as_numpy_iterator():
    
    batch_predictions = (model.predict(x) > 0.5).astype(int)
    all_imgs = []
    for i in range(len(batch_predictions)):
        if batch_predictions.flatten()[i] != y.flatten()[i].astype(int):
            print("Predicted: ",batch_predictions.flatten()[i], "Actual: ", y.flatten()[i].astype(int))
            
            fig = plt.figure(figsize=(25, 8))
            rows, cols = 3, 16
            use = x['axial']
            for i in range(1, cols*rows + 1):
                if i/cols == 1:
                    use = x['coronal']
                if i/cols == 2:
                    use = x['sagittal']
                    
                fig.add_subplot(rows, cols, i)
                
                plt.imshow(use[(i-1)%cols,:,:, 0])


            plt.show()

## IXI

In [4]:
import pandas as pd
import glob, os

ixi_root_dir = '/tf/shank/HDDLinux/Stanford/data/mriqc-shared/conformed/ixi/face/128'
ixi_deface_root_dir = '/tf/shank/HDDLinux/Stanford/data/mriqc-shared/conformed/ixi/deface/128'
cpaths = glob.glob(os.path.join(ixi_root_dir, '*.nii*'))
cpaths_deface = glob.glob(os.path.join(ixi_deface_root_dir, '*.nii*'))

# true_labels = pd.read_csv(os.path.join(test_root_dir, 'test1_images_gt.csv'))
# tl_dict = {}
# for i, row in enumerate(true_labels.values):
    
#     dataset, volume, label = row
#     l = 0 if 'deface' in label else 1
#     tl_dict[str(dataset) + '/' + str(volume)] = l
    
test_df_dict = {}
X, Y = [], []

for path in cpaths:
#     print(path)
    name = '/'.join(path.rsplit('/', 2)[-2:])
    X.append(path)
    Y.append(1)
    
for path in cpaths_deface:
#     print(path)
    name = '/'.join(path.rsplit('/', 2)[-2:])
    X.append(path)
    Y.append(0)


# print(X, Y)
import random

zipped = list(zip(X, Y))
random.shuffle(zipped)
_X, _Y = zip(*zipped)

test_df_dict['X'] = _X
test_df_dict['Y'] = _Y

df_test = pd.DataFrame(data=test_df_dict)
df_test.to_csv(os.path.join("/tf/shank/HDDLinux/Stanford/data/mriqc-shared/test_ixi/csv/testing.csv"), index=False)

## Convert test data to tfrecords

In [6]:
import random
import nobrainer
import os, sys
sys.path.append("..")
import numpy as np
import nibabel as nb
from glob import glob
from pathlib import Path
from shutil import *
import subprocess
from operator import itemgetter
import pandas as pd

test_root_dir = "/tf/shank/HDDLinux/Stanford/data/mriqc-shared/test_ixi"
csv_path = os.path.join(test_root_dir, "csv")
tf_records_dir = os.path.join(test_root_dir, "tfrecords")

os.makedirs(tf_records_dir, exist_ok=True)

test_csv_path = os.path.join(csv_path, "testing.csv")
test_paths = pd.read_csv(test_csv_path)["X"].values
test_labels = pd.read_csv(test_csv_path)["Y"].values
test_D = list(zip(test_paths, test_labels))
test_write_path = os.path.join(tf_records_dir, 'data-test_shard-{shard:03d}.tfrec')

nobrainer.tfrecord.write(
    features_labels=test_D,
    filename_template=test_write_path,
    examples_per_shard=3)

388/388 [==============================] - 83s 209ms/step


In [12]:
test_root_dir = '/tf/shank/HDDLinux/Stanford/data/mriqc-shared/test_ixi'
model_save_path = os.path.join(ROOTDIR_B, "model_save_dir_full")
tfrecords_path = os.path.join(test_root_dir, "tfrecords")
plane = "axial"
dataset_plane = get_dataset(
        file_pattern=os.path.join(tfrecords_path, "data-test_*"),
        n_classes=2,
        batch_size=16,
        volume_shape=(128, 128, 128),
        plane=plane,
        mode='test'
    )

print(dataset_plane)

<BatchDataset shapes: ((16, 128, 128, 1), (16, 1)), types: (tf.float32, tf.float32)>


## Inference

In [47]:
import sys, os
sys.path.append('../defacing')
from models.modelN import CombinedClassifier
import glob

# Tf packages
import tensorflow as tf
import pandas as pd
import numpy as np
from sklearn.utils import class_weight
from tensorflow.keras import backend as K
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import (
    ModelCheckpoint,
    LearningRateScheduler,
    TensorBoard,
)
from tensorflow.keras import metrics
from tensorflow.keras import losses

ROOTDIR_B = '/tf/shank/HDDLinux/Stanford/data/mriqc-shared/experiments/experiment_B/128'
ROOTDIR_A = '/tf/shank/HDDLinux/Stanford/data/mriqc-shared/experiments/experiment_A/128'
test_root_dir = '/tf/shank/HDDLinux/Stanford/data/mriqc-shared/test_ixi'

model_save_path = os.path.join(ROOTDIR_B, "model_save_dir_full")
tfrecords_path = os.path.join(test_root_dir, "tfrecords")
print("TFRECORDS: ", tfrecords_path)
weights_path = os.path.join(model_save_path, 'weights/combined/best-wts.h5')

planes = ['coronal'] #, 'coronal', 'sagittal']
for plane in planes:
    
    model = modelN.Submodel(
        input_shape=(128, 128),
        dropout=0.2,
        name=plane,
        include_top=True,
        weights=None,
        trainable=False,
    )
    
    print(os.path.join(model_save_path, plane, 'best-wts.h5'))
    
    model.load_weights(os.path.join(model_save_path, 'weights', plane, 'best-wts.h5'))
    
    dataset_plane = get_dataset(
        file_pattern=os.path.join(tfrecords_path, "data-test_*"),
        n_classes=2,
        batch_size=16,
        volume_shape=(128, 128, 128),
        plane=plane,
        mode='test',)
    
    METRICS = [
        metrics.BinaryAccuracy(name="accuracy"),
        metrics.Precision(name="precision"),
        metrics.Recall(name="recall"),
        metrics.AUC(name="auc"),
    ]
    
    model.summary()
    
    model.compile(
        loss=tf.keras.losses.binary_crossentropy,
        optimizer=Adam(learning_rate=1e-3),
        metrics=METRICS,
    )
    
#     results = model.evaluate(dataset_plane, batch_size=16)
    predictions = (model.predict(dataset_plane) > 0.5).astype(int)
    
# model = CombinedClassifier(
#     input_shape=(128, 128), dropout=0.4, wts_root=None, trainable=True
# )
# model.load_weights(os.path.abspath(weights_path))

# print(os.path.join(tfrecords_path, "data-test_*"))

# dataset_test = get_dataset(
#     file_pattern=os.path.join(tfrecords_path, "data-test_*"),
#     n_classes=2,
# #     n_slices = 24,
#     batch_size=16,
#     volume_shape=(128, 128, 128),
#     plane='combined',
#     mode='test'
# )

# print(dataset_test)

# METRICS = [
#             metrics.BinaryAccuracy(name="accuracy"),
#             metrics.Precision(name="precision"),
#             metrics.Recall(name="recall"),
#             metrics.AUC(name="auc"),
#         ]

# model.compile(
#     loss=tf.keras.losses.binary_crossentropy,
#     optimizer=Adam(learning_rate=1e-3),
#     metrics=METRICS,
# )

    
# results = model.evaluate(dataset_test, batch_size=16)
# predictions = (model.predict(dataset_test) > 0.5).astype(int)

TFRECORDS:  /tf/shank/HDDLinux/Stanford/data/mriqc-shared/test_ixi/tfrecords
/tf/shank/HDDLinux/Stanford/data/mriqc-shared/experiments/experiment_B/128/model_save_dir_full/coronal/best-wts.h5
Model: "model_20"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
coronal (InputLayer)         [(None, 128, 128, 1)]     0         
_________________________________________________________________
conv2d_108 (Conv2D)          (None, 128, 128, 8)       80        
_________________________________________________________________
batch_normalization_108 (Bat (None, 128, 128, 8)       32        
_________________________________________________________________
activation_108 (Activation)  (None, 128, 128, 8)       0         
_________________________________________________________________
conv2d_109 (Conv2D)          (None, 128, 128, 8)       584       
________________________________________________________________

In [48]:
print(len(predictions.flatten()))

1152


In [49]:
import matplotlib.pyplot as plt
corr_pred_map = {}
corr = 0
incorr = 0
for x, y in dataset_plane.as_numpy_iterator():
    
    batch_predictions = (model.predict(x) > 0.5).astype(int)
    all_imgs = []
    for i in range(len(batch_predictions)):
        if batch_predictions.flatten()[i] != y.flatten()[i].astype(int):
            incorr += 1
            print("Predicted: ",batch_predictions.flatten()[i], "Actual: ", y.flatten()[i].astype(int))
        else:
            corr += 1
            
#             fig = plt.figure(figsize=(25, 8))
#             rows, cols = 3, 16
            
#             for i in range(1, cols*rows + 1):
# #                 if i/cols == 1:
# #                     use = x['coronal']
# #                 if i/cols == 2:
# #                     use = x['sagittal']
                    
#                 fig.add_subplot(rows, cols, i)
                
#                 plt.imshow(use[(i-1)%cols,:,:, 0])


#             plt.show()

Predicted:  0 Actual:  1
Predicted:  0 Actual:  1
Predicted:  0 Actual:  1
Predicted:  0 Actual:  1
Predicted:  0 Actual:  1
Predicted:  0 Actual:  1
Predicted:  0 Actual:  1
Predicted:  0 Actual:  1
Predicted:  0 Actual:  1
Predicted:  0 Actual:  1
Predicted:  0 Actual:  1
Predicted:  0 Actual:  1
Predicted:  0 Actual:  1
Predicted:  0 Actual:  1
Predicted:  0 Actual:  1
Predicted:  0 Actual:  1
Predicted:  0 Actual:  1
Predicted:  0 Actual:  1
Predicted:  0 Actual:  1
Predicted:  0 Actual:  1
Predicted:  0 Actual:  1
Predicted:  0 Actual:  1
Predicted:  0 Actual:  1


/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:16: RuntimeWarning: More than 20 figures have been opened. Figures created through the pyplot interface (`matplotlib.pyplot.figure`) are retained until explicitly closed and may consume too much memory. (To control this warning, see the rcParam `figure.max_open_warning`).
  app.launch_new_instance()


Predicted:  0 Actual:  1
Predicted:  0 Actual:  1
Predicted:  0 Actual:  1
Predicted:  0 Actual:  1
Predicted:  0 Actual:  1
Predicted:  0 Actual:  1
Predicted:  0 Actual:  1
Predicted:  0 Actual:  1
Predicted:  0 Actual:  1
Predicted:  0 Actual:  1
Predicted:  0 Actual:  1
Predicted:  0 Actual:  1
Predicted:  0 Actual:  1
Predicted:  0 Actual:  1
Predicted:  0 Actual:  1
Predicted:  0 Actual:  1
Predicted:  0 Actual:  1
Predicted:  0 Actual:  1
Predicted:  0 Actual:  1
Predicted:  0 Actual:  1
Predicted:  0 Actual:  1
Predicted:  0 Actual:  1
Predicted:  0 Actual:  1
Predicted:  0 Actual:  1
Predicted:  0 Actual:  1
Predicted:  0 Actual:  1
Predicted:  0 Actual:  1
Predicted:  0 Actual:  1
Predicted:  0 Actual:  1
Predicted:  0 Actual:  1
Predicted:  0 Actual:  1
Predicted:  0 Actual:  1
Predicted:  0 Actual:  1
Predicted:  0 Actual:  1
Predicted:  0 Actual:  1
Predicted:  0 Actual:  1
Predicted:  0 Actual:  1
Predicted:  0 Actual:  1
Predicted:  0 Actual:  1
Predicted:  0 Actual:  1


Predicted:  0 Actual:  1
Predicted:  0 Actual:  1
Predicted:  0 Actual:  1
Predicted:  0 Actual:  1
Predicted:  0 Actual:  1
Predicted:  0 Actual:  1
Predicted:  0 Actual:  1
Predicted:  0 Actual:  1
Predicted:  0 Actual:  1
Predicted:  0 Actual:  1
Predicted:  0 Actual:  1
Predicted:  0 Actual:  1
Predicted:  0 Actual:  1
Predicted:  0 Actual:  1
Predicted:  0 Actual:  1
Predicted:  0 Actual:  1
Predicted:  0 Actual:  1
Predicted:  0 Actual:  1
Predicted:  0 Actual:  1
Predicted:  0 Actual:  1
Predicted:  0 Actual:  1
Predicted:  0 Actual:  1
Predicted:  0 Actual:  1
Predicted:  0 Actual:  1
Predicted:  0 Actual:  1
Predicted:  0 Actual:  1
Predicted:  0 Actual:  1
Predicted:  0 Actual:  1
Predicted:  0 Actual:  1
Predicted:  0 Actual:  1
Predicted:  0 Actual:  1
Predicted:  0 Actual:  1
Predicted:  0 Actual:  1
Predicted:  0 Actual:  1
Predicted:  0 Actual:  1
Predicted:  0 Actual:  1
Predicted:  0 Actual:  1
Predicted:  0 Actual:  1
Predicted:  0 Actual:  1
Predicted:  0 Actual:  1


<Figure size 1800x576 with 0 Axes>

...

<Figure size 1800x576 with 0 Axes>

<Figure size 1800x576 with 0 Axes>

<Figure size 1800x576 with 0 Axes>

<Figure size 1800x576 with 0 Axes>

<Figure size 1800x576 with 0 Axes>

In [50]:
print(corr, incorr)

580 572
